Hello guys, this kernel i wanted to do some kind of Multiple Linear Regression, but i failed because the dataset doenst have a normal distribution. Maybe, i'm a newb yet, but currently is the only machile learning method i know.

So in this dataset i cleaned some values, transformed data, maybe i could create some other variables.

So If you like this notebook. Please upvote!

Feel free to give suggestions, advices and tips!

Thanks!

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Getting some early infos about dataset

In [ ]:
data = pd.read_csv('/kaggle/input/udemy-courses/udemy_courses.csv')
data.head()

In [ ]:
data.shape

## Dataset has 3683 lines and 12 columns

## As you see below, we have 3683 non null values... So no missing data

In [ ]:
data.info()

## We can see that the column published_timestamp, price and content_duration isn't in the properly type

Let's start with content_duration

In [ ]:
data['content_duration'].unique()

## Lets filter only data with hours and hour in the dataset

In [ ]:
data = data[data['content_duration'].str.contains('hour')]

In [ ]:
data.head()

In [ ]:
def ajuste_hour(hour):
    hora = hour.replace('hour','')
    hora1 = hora.replace('s','')
    output = float(hora1)
    return output

In [ ]:
data['content_duration'] = data['content_duration'].apply(ajuste_hour)

Checking if data changed

In [ ]:
data.info()

Yes! Changed to float

## Now let's clean the price

In [ ]:
data['price'].unique()

We have free courses

In [ ]:
def free_course(course):
    price = course.replace('Free','0')
    price1 = float(price)
    return price1

In [ ]:
data['price'] = data['price'].apply(free_course)

In [ ]:
data.info()

Now price has only float numbers, as we expected

## Now we have to fix the timestamp

In [ ]:
data['published_timestamp']= pd.to_datetime(data['published_timestamp'], format = '%Y-%m-%dT%H:%M:%SZ')

In [ ]:
data.head()

In [ ]:
data.info()

Splitting to date

In [ ]:
data['date'] = pd.to_datetime(data['published_timestamp'].dt.date, format = '%Y-%m-%d')
data.head()

## Now that we finally cleaned the dataset, lets try to visualize somethings.
## We will start with categorical data

In [ ]:
categorical = [var for var in data.columns if data[var].dtype=='O']

print('There are {} categorical variables\n'.format(len(categorical)))

print('The categorical variables are :\n\n', categorical)

In [ ]:
data['is_paid'].unique(), data['level'].unique(), data['subject'].unique()

## We can see that is_paid has variations of True and False, we have to fix it

In [ ]:
data['is_paid'] = data['is_paid'].replace({'TRUE': 'True', 'FALSE':'False'})
data['is_paid'].unique()

## Now that we finally changed this erros lets now plot some views

In [ ]:
# so we have 3 categorical data, we can explore these data here
# view the frequency 
for var in categorical:
    print(data[var].value_counts()/np.float(len(data)))

## Lets visualize some data


In [ ]:
ax = data['is_paid'].value_counts().plot(kind ='bar',
                                        figsize = (10,8))

ax.set_title('Almost every course in udemy is paid', fontsize = 22)
ax.set_ylabel('Amount of courses', fontsize = 20)
ax.set_xlabel('Paid', fontsize = 20)

## Majority of courses is paid..

In [ ]:
ax = data['subject'].value_counts().plot(kind ='bar',
                                        figsize = (10,8),
                                        color = 'g',
                                        width = 0.6,
                                        alpha = 0.6)

ax.set_title('Huge focus on web development and Business', fontsize = 22)
ax.set_ylabel('Amount of courses', fontsize = 20)
ax.set_xlabel('Courses', fontsize = 20)

## A lot of courses are created for web development and business finance, perhaps udemy has a huge demand on theses general themes

In [ ]:
ax = data['level'].value_counts().plot(kind ='bar',
                                        figsize = (10,8),
                                      width = 0.8,
                                      alpha= 0.7)

ax.set_title('Designed for everybody!', fontsize = 22)
ax.set_ylabel('Amount of courses', fontsize = 20)
ax.set_xlabel('Levels', fontsize = 20)

## These one i found really interesting, because it's designed for everybody!
## If you want to learn, you don't have excuses

## Let's see how its distribuited this data


In [ ]:
grouped = data.groupby(['level','subject'])

In [ ]:
grouped_pct = grouped['course_id']

In [ ]:
grouped_pct.agg('describe')

## Lets explore numerical data

In [ ]:
# now we are going to explore numerical data.
numerical = [var for var in data.columns if data[var].dtype!='O']

print('There are {} numerical variables\n'.format(len(numerical)))

print('The numerical variables are :', numerical)

In [ ]:
data.corr().round(4)

## Checking the variable price

In [ ]:
ax = sns.boxplot(data['price'], orient = 'v', width = 0.3)
ax.figure.set_size_inches(12,6)
ax.set_ylabel('Price', fontsize = 20)
ax.set_title('Distribuition from courses', fontsize  = 24)

## Not looking like a normal distribution

In [ ]:
ax = sns.distplot(data['price'])
ax.figure.set_size_inches(12,6)
ax.set_xlabel('Prices')
ax.set_title('Distribution', fontsize = 22)

## Checking boxplot from price with other variables

In [ ]:
ax = sns.boxplot(x = 'level',y = 'price', data = data, orient = 'v', width = 0.3)
ax.figure.set_size_inches(12,6)
ax.set_ylabel('Price', fontsize = 20)
ax.set_title('Distribution of price with level', fontsize  = 24)
ax.set_xlabel('Level', fontsize= 20)

In [ ]:
ax = sns.boxplot(x = 'subject',y = 'price', data = data, orient = 'v', width = 0.3)
ax.figure.set_size_inches(12,6)
sns.set_palette('CMRmap')
ax.set_ylabel('Price', fontsize = 20)
ax.set_title('Distribution of price with subjects', fontsize  = 24)
ax.set_xlabel('Subjects', fontsize= 20)

In [ ]:
ax = sns.pairplot(data, y_vars = 'price', x_vars = ['num_subscribers', 'num_reviews', 'num_lectures','content_duration'], height = 5, kind = 'reg')
ax.fig.suptitle('Scatter plot with variables', fontsize=20, y=1.05)
ax

## Transforming to log

In [ ]:
data['log_price'] = np.log(data['price']+1)
data['log_num_subscribers'] = np.log(data['num_subscribers'])
data['log_num_reviews'] = np.log(data['num_reviews'] + 1)
data['log_num_lectures'] = np.log(data['num_lectures'])
data['log_content_duration'] = np.log(data['content_duration'])

In [ ]:
data.head()

In [ ]:
## Filtering only paid courses
data_new = data[data['is_paid'] == 'True']

In [ ]:
data_new.head()

## Here i tried to check if the price variable would became a normal distribution

In [ ]:
ax = sns.distplot(data_new['log_price'], bins =15)
ax.figure.set_size_inches(12,6)
ax.set_xlabel('Price')
ax.set_title('Distribution of log_price', fontsize = 22)

## Well i give up haha! Let's clean some outliers

In [ ]:
group_subject = data_new.groupby('subject')['price']

In [ ]:
type(group_subject)

In [ ]:
group_subject.groups

In [ ]:
group_subject.mean()

In [ ]:
Q1 = group_subject.quantile(.25)
Q3 = group_subject.quantile(.75)
IIQ = Q3 - Q1 #intervalo interquartilico
lower_limit = Q1 - 1.5*IIQ
upper_limit = Q3 + 1.5*IIQ

In [ ]:
data_new1 = pd.DataFrame()

for subject in group_subject.groups.keys():
    is_subject = data_new['subject'] == subject
    is_limit = (data_new['price'] >= lower_limit[subject]) & (data_new['price'] <= upper_limit[subject])
    selection = is_subject & is_limit
    data_select = data_new[selection]
    data_new1 = pd.concat([data_new1, data_select])

## Plotting now.. without outliers

In [ ]:
ax = sns.boxplot(x = 'subject',y = 'price', data = data_new1, orient = 'v', width = 0.3)
ax.figure.set_size_inches(12,6)
sns.set_palette('CMRmap')
ax.set_ylabel('Preço', fontsize = 20)
ax.set_title('Preço dos cursos', fontsize  = 24)
ax.set_xlabel('Subjects', fontsize= 20)

## Checking the distribution

In [ ]:
ax = sns.distplot(data_new1['log_price'], bins =15)
ax.figure.set_size_inches(12,6)
ax.set_xlabel('Price')
ax.set_title('Distribution of price', fontsize = 22)